# LLM as Player

In [2]:
!pip install openai ollama -q


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [4]:
import logging, sys, os

import matplotlib.pyplot as plt
from openai import OpenAI
from dotenv import load_dotenv

from maverick import (
    Game,
    Player,
    ActionType,
    GameState,
    PlayerLike,
    PlayerAction,
    PlayerState,
)
from maverick.players import FoldBot, CallBot, AggressiveBot
from maverick.utils import estimate_strongest_hand, score_hand

_ = load_dotenv(override=True)

In [5]:
# OLLAMA_BASE_URL = "http://localhost:11434/v1"
# model_name = "llama3.2"
# llm = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')

model_name = "gpt-5-nano"
api_key = os.getenv('OPENAI_API_KEY')
llm = OpenAI(api_key=api_key)

In [6]:
response = llm.responses.parse(
    model=model_name, 
    input=[{"role": "user", "content": "Are you ready to play poker?"}]
)
response.output_text

'Yes, I’m ready. Do you want to play Texas Hold’em? I can run a quick heads-up game with standard rules.\n\nSuggested setup (we can adjust if you prefer):\n- No-limit Texas Hold’em\n- Starting chips: 1000 each\n- Blinds: 10/20\n- Betting actions: fold, check, bet <amount>, call, raise <amount>\n\nIf this sounds good, I’ll deal the first hand and you can act. Just say “deal” or tell me any changes you want (variant, stakes, number of players).'

In [7]:
system_message = "You are a poker player playing a game of Texas Hold'em poker. Make decisions based on the game state and your cards."


class LLM(Player):

    def decide_action(
        self,
        game_state: GameState,
        valid_actions: list[ActionType],
        min_raise: int
    ) -> PlayerAction:
        print(f"\n{self.name} is requested to take action.")
        
        # Information related to the game itself
        print("     Current street:", game_state.street.name)
        print("     Button position:", game_state.button_position)
        print("     Current pot:", game_state.pot)
        print("     Current bet:", game_state.current_bet)
        print("     Small blind:", game_state.small_blind)
        print("     Big blind:", game_state.big_blind)
        print("     Community cards:", game_state.community_cards)
        
        # Information related to the player
        print("     Player stack:", self.state.stack)
        print("     Current player bet:", self.state.current_bet)
        print("     Private cards:", self.state.holding.cards)
        
        # Information related to the actions available for the player
        print("     Valid actions:", [v.name for v in valid_actions])
        print("     Minimum raise:", min_raise)

        # Private and community cards
        private_cards = self.state.holding.cards
        community_cards = game_state.community_cards
        print("     Private cards:", private_cards)
        print("     Community cards:", community_cards)
        
        # Estimate the strongest possible hand of the player, considering
        # all community cards.
        strongest_hand, strongest_hand_prob = estimate_strongest_hand(
            private_cards,
            community_cards,
            n_min_private=0,
            n_simulations=1000,
            n_players=len(game_state.get_players_in_hand())
        )
        strongest_hand_type, strongest_hand_score = score_hand(strongest_hand)
        print("     Strongest hand:", strongest_hand)
        print("     Strongest hand type:", strongest_hand_type.name)
        print("     Strongest hand score:", strongest_hand_score)
        print("     Estimated strength:", strongest_hand_prob)
        
        player_message = f"""
You are playing a game of Texas Hold'em poker. Respond with your next action based on the following game 
state information:

    - Your id is {self.id} and your name is {self.name}.
    - The current street is {game_state.street.name}.
    - The community cards are: {community_cards}
    - The pot size is: {game_state.pot}
    - The current bet to call is: {game_state.current_bet}
    - The small blind is: {game_state.small_blind}
    - The big blind is: {game_state.big_blind}
    - Your stack size is: {self.state.stack}
    - The valid actions you can take are: {', '.join([v.name for v in valid_actions])}
    - The minimum raise amount is: {min_raise}
    - Your cards are: {private_cards}

    - Based on a Monte-Carlo simulation, your strongest possible hand is {strongest_hand} ({strongest_hand_type.name}) with an estimated 
      strength of {strongest_hand_prob:.2f}. The estimated strength represents the probability of having the best 
      hand against the other players still in the game.

Rules:
    - You **MUST** choose one of the following actions: {', '.join([v.name for v in valid_actions])}.
    - If you choose to RAISE, you must specify an amount greater than or equal to the minimum raise.
    - You only need to specify an amount if your action is RAISE or BET.
"""
        
        try:
            response = llm.responses.parse(
                model=model_name,
                input=[
                    {"role": "system", "content": system_message},
                    {"role": "user", "content": player_message},
                ],
                text_format=PlayerAction,
            )
            
            action = response.output_parsed
        except Exception as e:
            logging.error(f"LLMPlayer {self.id} failed to decide action: {e}")
            action = PlayerAction(player_id=self.id, action_type=ActionType.FOLD)
        
        print(f"     Player decides to {action.action_type.name}.\n")
        sys.stdout.flush()  # Ensure all prints are output before returning
        return action

In [10]:
# Configure logging such that we only get the log messages of the game
logging.basicConfig(
    level=logging.DEBUG,
    format="[%(levelname)s] %(name)s: %(message)s",
)
logging.getLogger().setLevel(logging.WARNING)  # Set root logger to WARNING
logging.getLogger("maverick").setLevel(logging.DEBUG)  # Only maverick logs at DEBUG

# Create game with blinds
game = Game(small_blind=10, big_blind=20, max_hands=40)

# Create and add players with different strategies
players: list[PlayerLike] = [
    CallBot(id="p1", name="CallBot", state=PlayerState(stack=1000, seat=0)),
    AggressiveBot(id="p2", name="AggroBot", state=PlayerState(stack=1000, seat=1)),
    LLM(id="p3", name=model_name, state=PlayerState(stack=1000, seat=2)),
    FoldBot(id="p4", name="FoldBot", state=PlayerState(stack=1000, seat=3)),
]

for player in players:
    game.add_player(player)
    
game.start()

2026-01-10 23:50:19,441 [INFO] maverick: Player CallBot joined the game.
2026-01-10 23:50:19,442 [INFO] maverick: Player AggroBot joined the game.
2026-01-10 23:50:19,442 [INFO] maverick: Player gpt-5-nano joined the game.
2026-01-10 23:50:19,442 [INFO] maverick: Player FoldBot joined the game.
2026-01-10 23:50:19,443 [INFO] maverick: Game started.

2026-01-10 23:50:19,443 [INFO] maverick: ============================== Hand 1 ==============================

2026-01-10 23:50:19,443 [INFO] maverick: PRE_FLOP | Dealing hole cards. Button: CallBot
2026-01-10 23:50:19,444 [INFO] maverick: PRE_FLOP | Posting small blind of 10 by player AggroBot. Remaining stack: 990
2026-01-10 23:50:19,445 [INFO] maverick: PRE_FLOP | Posting big blind of 20 by player gpt-5-nano. Remaining stack: 980
2026-01-10 23:50:19,445 [INFO] maverick: PRE_FLOP | Player FoldBot folds.
2026-01-10 23:50:19,446 [INFO] maverick: PRE_FLOP | Current pot: 30 | Current bet: 20
2026-01-10 23:50:19,446 [INFO] maverick: PRE_FLOP |


gpt-5-nano is requested to take action.
     Current street: PRE_FLOP
     Button position: 0
     Current pot: 90
     Current bet: 50
     Small blind: 10
     Big blind: 20
     Community cards: []
     Player stack: 980
     Current player bet: 20
     Private cards: [10♠, 5♠]
     Valid actions: ['FOLD', 'CALL', 'RAISE', 'ALL_IN']
     Minimum raise: 70
     Private cards: [10♠, 5♠]
     Community cards: []
     Strongest hand: [10♠, 5♠]
     Strongest hand type: HIGH_CARD
     Strongest hand score: 100.1005
     Estimated strength: 0.307
     Player decides to FOLD.



2026-01-10 23:50:47,121 [INFO] maverick: PRE_FLOP | Player gpt-5-nano folds.
2026-01-10 23:50:47,121 [INFO] maverick: PRE_FLOP | Current pot: 90 | Current bet: 50
2026-01-10 23:50:47,122 [INFO] maverick: PRE_FLOP | Player CallBot calls with amount 30. Remaining stack: 950.
2026-01-10 23:50:47,122 [INFO] maverick: PRE_FLOP | Current pot: 120 | Current bet: 50
2026-01-10 23:50:47,122 [INFO] maverick: PRE_FLOP | Betting round complete

2026-01-10 23:50:47,123 [INFO] maverick: FLOP | Dealt flop. Community cards: ['5♦', 'J♠', '10♥']
2026-01-10 23:50:47,123 [INFO] maverick: FLOP | Player AggroBot bets amount 40. Remaining stack: 910.
2026-01-10 23:50:47,123 [INFO] maverick: FLOP | Current pot: 160 | Current bet: 40
2026-01-10 23:50:47,123 [INFO] maverick: FLOP | Player CallBot calls with amount 40. Remaining stack: 910.
2026-01-10 23:50:47,123 [INFO] maverick: FLOP | Current pot: 200 | Current bet: 40
2026-01-10 23:50:47,124 [INFO] maverick: FLOP | Betting round complete

2026-01-10 23:50:47


gpt-5-nano is requested to take action.
     Current street: PRE_FLOP
     Button position: 1
     Current pot: 90
     Current bet: 40
     Small blind: 10
     Big blind: 20
     Community cards: []
     Player stack: 970
     Current player bet: 10
     Private cards: [2♠, A♦]
     Valid actions: ['FOLD', 'CALL', 'RAISE', 'ALL_IN']
     Minimum raise: 60
     Private cards: [2♠, A♦]
     Community cards: []
     Strongest hand: [2♠, A♦]
     Strongest hand type: HIGH_CARD
     Strongest hand score: 100.1402
     Estimated strength: 0.309
     Player decides to FOLD.



2026-01-10 23:51:10,971 [INFO] maverick: PRE_FLOP | Player gpt-5-nano folds.
2026-01-10 23:51:10,974 [INFO] maverick: PRE_FLOP | Current pot: 90 | Current bet: 40
2026-01-10 23:51:10,980 [INFO] maverick: PRE_FLOP | Player FoldBot folds.
2026-01-10 23:51:10,985 [INFO] maverick: PRE_FLOP | Current pot: 90 | Current bet: 40
2026-01-10 23:51:10,985 [INFO] maverick: PRE_FLOP | Player CallBot calls with amount 20. Remaining stack: 790.
2026-01-10 23:51:10,986 [INFO] maverick: PRE_FLOP | Current pot: 110 | Current bet: 40
2026-01-10 23:51:10,993 [INFO] maverick: PRE_FLOP | Betting round complete

2026-01-10 23:51:10,997 [INFO] maverick: FLOP | Dealt flop. Community cards: ['K♣', '8♣', '7♠']
2026-01-10 23:51:10,998 [INFO] maverick: FLOP | Player CallBot checks.
2026-01-10 23:51:10,998 [INFO] maverick: FLOP | Current pot: 110 | Current bet: 0
2026-01-10 23:51:10,999 [INFO] maverick: FLOP | Player AggroBot bets amount 40. Remaining stack: 1110.
2026-01-10 23:51:10,999 [INFO] maverick: FLOP | Cur

...

2026-01-11 00:17:28,098 [INFO] maverick: PRE_FLOP | Player gpt-5-nano calls with amount 30. Remaining stack: 50.
2026-01-11 00:17:28,100 [INFO] maverick: PRE_FLOP | Current pot: 120 | Current bet: 50
2026-01-11 00:17:28,100 [INFO] maverick: PRE_FLOP | Player CallBot calls with amount 30. Remaining stack: 1980.
2026-01-11 00:17:28,100 [INFO] maverick: PRE_FLOP | Current pot: 150 | Current bet: 50
2026-01-11 00:17:28,101 [INFO] maverick: PRE_FLOP | Betting round complete

2026-01-11 00:17:28,101 [INFO] maverick: FLOP | Dealt flop. Community cards: ['10♥', '8♦', '7♣']
2026-01-11 00:17:28,101 [INFO] maverick: FLOP | Player AggroBot bets amount 40. Remaining stack: 900.
2026-01-11 00:17:28,102 [INFO] maverick: FLOP | Current pot: 190 | Current bet: 40



gpt-5-nano is requested to take action.
     Current street: FLOP
     Button position: 0
     Current pot: 190
     Current bet: 40
     Small blind: 10
     Big blind: 20
     Community cards: [10♥, 8♦, 7♣]
     Player stack: 50
     Current player bet: 0
     Private cards: [Q♥, 5♦]
     Valid actions: ['FOLD', 'CALL', 'ALL_IN']
     Minimum raise: 60
     Private cards: [Q♥, 5♦]
     Community cards: [10♥, 8♦, 7♣]
     Strongest hand: [Q♥, 5♦, 10♥, 8♦, 7♣]
     Strongest hand type: HIGH_CARD
     Strongest hand score: 100.1210080705
     Estimated strength: 0.014
     Player decides to FOLD.



2026-01-11 00:17:50,419 [INFO] maverick: FLOP | Player gpt-5-nano folds.
2026-01-11 00:17:50,420 [INFO] maverick: FLOP | Current pot: 190 | Current bet: 40
2026-01-11 00:17:50,421 [INFO] maverick: FLOP | Player CallBot calls with amount 40. Remaining stack: 1940.
2026-01-11 00:17:50,422 [INFO] maverick: FLOP | Current pot: 230 | Current bet: 40
2026-01-11 00:17:50,422 [INFO] maverick: FLOP | Betting round complete

2026-01-11 00:17:50,423 [INFO] maverick: TURN | Dealt turn. Community cards: ['10♥', '8♦', '7♣', '9♥']
2026-01-11 00:17:50,424 [INFO] maverick: TURN | Player AggroBot bets amount 40. Remaining stack: 860.
2026-01-11 00:17:50,425 [INFO] maverick: TURN | Current pot: 270 | Current bet: 40
2026-01-11 00:17:50,425 [INFO] maverick: TURN | Player CallBot calls with amount 40. Remaining stack: 1900.
2026-01-11 00:17:50,426 [INFO] maverick: TURN | Current pot: 310 | Current bet: 40
2026-01-11 00:17:50,426 [INFO] maverick: TURN | Betting round complete

2026-01-11 00:17:50,427 [INFO]


gpt-5-nano is requested to take action.
     Current street: PRE_FLOP
     Button position: 1
     Current pot: 90
     Current bet: 40
     Small blind: 10
     Big blind: 20
     Community cards: []
     Player stack: 40
     Current player bet: 10
     Private cards: [6♣, 6♥]
     Valid actions: ['FOLD', 'CALL', 'ALL_IN']
     Minimum raise: 60
     Private cards: [6♣, 6♥]
     Community cards: []
     Strongest hand: [6♣, 6♥]
     Strongest hand type: PAIR
     Strongest hand score: 206
     Estimated strength: 0.546


KeyboardInterrupt: 

In [ ]:
for player in players:
    print(f"{player.name} - Stack: {player.state.stack}")

CallBot - Stack: 580
AggroBot - Stack: 1455
gpt-5-nano - Stack: 995
FoldBot - Stack: 970
